エミュレータ
-



In [267]:
import os
import torch
import json
import numpy as np

OUTPUT_DIR = "output_train_3"
os.makedirs(OUTPUT_DIR, exist_ok=True)

保存
-
Generatorの保存済みパラメータをint8で保存

In [268]:

# def export_generator(netG_class,pth_path, OUTPUT_DIR,folder_name="G_q17_list",nz=100, ngf=128, device="cpu"):
#     save_dir = os.path.join(OUTPUT_DIR, folder_name)
#     os.makedirs(save_dir, exist_ok=True)

#     netG = netG_class(nz=nz, ngf=ngf).to(device)
#     state_dict = torch.load(pth_path, map_location=device)
#     netG.load_state_dict(state_dict)
#     netG.eval()

#     meta = {}

#     conv_idx = 0
#     for layer in netG.net:
#         if isinstance(layer, torch.nn.ConvTranspose2d):
#             name = f"net{conv_idx}"
#             conv_idx += 1

#             w = layer.weight.detach()
#             w_list = float_to_int8_list(w)

#             # JSON保存
#             json_path = os.path.join(save_dir, f"{name}_weight_q17_list.json")
#             with open(json_path, "w", encoding="utf-8") as f:
#                 json.dump(w_list, f)

#             # TXT保存
#             txt_path = os.path.join(save_dir, f"{name}_weight_q17_list.txt")
#             with open(txt_path, "w", encoding="utf-8") as f:
#                 f.write(f"{name} weight (Q1.7 int8)\n")
#                 f.write(f"shape = [out_ch][in_ch][4][4]\n")
#                 f.write("value = int8 (real=value/128)\n\n")

#                 # w_list [out_ch][in_ch][kH][kW]
#                 for out_ch_id, oc_block in enumerate(w_list):
#                     f.write(f"=== out_ch {out_ch_id} ===\n")
#                     for in_ch_id, kernel in enumerate(oc_block):
#                         f.write(f" -- in_ch {in_ch_id} --\n")
#                         for row in kernel:
#                             f.write("".join(f"{v:4d}" for v in row) + "\n")
#                         f.write("\n")

#             meta[name] = {"shape":[len(w_list),len(w_list[0]),4,4],"json":os.path.basename(json_path),"txt":os.path.basename(txt_path)}
#     # メタ情報
#     meta_path = os.path.join(save_dir, "meta.json")
#     with open(meta_path, "w", encoding="utf-8") as f:
#         json.dump(meta, f, indent=2)
    
    # return save_dir

            

In [269]:
# import torch
# from torch import nn, optim
# class Generator(nn.Module):
#     def __init__(self, nz=100, ngf=128):
#         super().__init__()
#         self.nz = nz

#         self.net = nn.Sequential(
#             # [B,nz,1,1] → [B,ngf*4,4,4]
#             nn.ConvTranspose2d(nz, ngf*4, 4, 1, 0, bias=False),　　　
#             nn.ReLU(True),

#             # [B,ngf*4,4,4] → [B,ngf*2,8,8]
#             nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
#             nn.ReLU(True),

#             # [B,ngf*2,8,8] → [B,ngf,16,16]
#             nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
#             nn.ReLU(True),

#             # [B,ngf,16,16] -> [B,1,32,32]
#             nn.ConvTranspose2d(ngf, 1, 4, 2, 1, bias=False),
#             nn.Tanh()  # [-1,1]
#         )

   
#     def forward(self,z):
#          # [B,100] →　[B,100,1,1]
#         if z.dim() == 2:
#             z = z.view(z.size(0),z.size(1),1,1)
#         return self.net(z)

In [270]:
# pth_path = os.path.join(OUTPUT_DIR, "netG_float32.pth")

# save_dir = export_generator(
#     netG_class=Generator,
#     pth_path=pth_path,
#     OUTPUT_DIR=OUTPUT_DIR,
#     folder_name="G_q17_list",
#     nz=100,
#     ngf=128,
#     device="cpu"
# )

# print("saved to:",save_dir)

int　→　Q1.7（8bit）   2進数表現

In [271]:
# def int8_to_bin8(v:int) -> str:
#     return format(v & 0xFF, "08b")

# def json_to_bin(save_dir: str, out_folder_name:str="bin_txt"):
#     meta_path = os.path.join(save_dir, "meta.json")
#     with open(meta_path, "r", encoding="utf-8") as f:
#         meta = json.load(f)

#     out_dir = os.path.join(save_dir, out_folder_name)
#     os.makedirs(out_dir, exist_ok=True)

#     for name, info in meta.items():
#         json_path = os.path.join(save_dir, info["json"])

#         with open(json_path, "r", encoding="utf-8") as f:
#             w_list = json.load(f)

#         txt_path = os.path.join(out_dir, f"{name}_weight_q17_bin.txt")
#         with open(txt_path, "w", encoding="utf-8") as ftxt:
#             ftxt.write(f"{name} weight (Q1.7) as 8-bit two's complement binary\n")
#             ftxt.write("shape = [out_ch][in_ch][4][4]\n")
#             ftxt.write("each value is 8 bit: b7..b0\n\n")

#             for out_ch_id,oc_block in enumerate(w_list):
#                 ftxt.write(f"=== out_ch {out_ch_id} === \n")
#                 for in_ch_id, kernel in enumerate(oc_block):
#                     ftxt.write(f"-- in_ch {in_ch_id} -- \n")
#                     for row in kernel:
#                         ftxt.write(" ".join(int8_to_bin8(v) for v in row) + "\n")
#                     ftxt.write("\n")
#     return out_dir

In [272]:
# save_dir = os.path.join(OUTPUT_DIR, "G_q17_list")
# bin_txt_dir = json_to_bin(save_dir=save_dir)
# print("save to:",bin_txt_dir)

NNのパラメータ確認用 float

In [273]:
# import torch

# state = torch.load("output_train_3/netG_float32.pth", map_location="cpu")

# print(type(state))
# print(state.keys())


# w = state["net.0.weight"]

# # ① net.0.weight の「一つ目の要素」
# print("net.0.weight[0,0,:,:] =")
# print(w[0, 0])

# # ② net.0.weight 全体の最小・最大
# print("net.0.weight min =", w.min().item())
# print("net.0.weight max =", w.max().item())



固定小数点の実装
-
符号1bit、整数部7bit、少数部8bit Q7.8 (16bit)   範囲[-128,127.996]   分解能　1/256


「floatで-128.0 ～ +127.996の計算範囲」を「int16の-32768 ～ 32767の範囲」にマッピング

127.996(float) = 32767(int16)      

「int16の時の値」＝　「実数値(floatの値)」　×　2^8

In [274]:
Q_FRAC = 8
Q_SCALE = 1 << Q_FRAC # 256 (少数部 2^8)
Q_MIN = -32768 # int16 
Q_MAX = 32767

def float_to_int16_torch(x:torch.Tensor) -> torch.Tensor:
    x_q = torch.round(x*Q_SCALE)
    x_q = torch.clamp(x_q, Q_MIN, Q_MAX)
    return x_q.to(torch.int16)

def float_to_int16_np(x:np.ndarray) -> np.ndarray:
    x_q = np.rint(x*Q_SCALE)
    x_q = np.clip(x_q, Q_MIN, Q_MAX)
    return x_q.astype(np.int16)

def int16_to_float_torch(x_q: torch.Tensor) -> torch.Tensor:
    return x_q.to(torch.float32) / Q_SCALE

def int16_to_float_np(x_q: np.ndarray) -> np.ndarray:
    return x_q.astype(np.int16) / Q_SCALE

### v1v2の読み込み

In [275]:
path = "output_train_3/v1v2.pth"

data = torch.load(path, map_location="cpu")
print(type(data))
print(data.keys())

v1 = data["v1"]
v2 = data["v2"]

v1 = float_to_int16_torch(v1)
v2 = float_to_int16_torch(v2)
# print(v1)


<class 'dict'>
dict_keys(['v1', 'v2'])


入力生成
-


In [ ]:

z = torch.randn(100)
print(z)

# float張り付け
# #z = [-0.0529,  1.3876, -0.1987, -0.6410, -0.4712, -1.2671,  1.5316, -0.3863,
#         -1.3278, -0.3035, -1.4634,  0.3605,  1.0053,  1.2927, -0.5330, -0.8077,
#          0.7029, -1.9641, -0.4360,  0.3952,  0.4549, -1.0385,  1.2419, -0.1570,
#         -0.3820, -0.4848, -1.7011, -0.4536, -0.4430,  1.1325,  0.7088, -0.3032,
#         -0.0947,  1.1591, -1.6729,  2.2497,  0.7753,  0.6680,  0.4857,  0.1887,
#          1.6274,  0.8200, -2.4146, -0.4427, -0.9399, -1.0022,  0.5288,  0.4581,
#          1.3886,  1.1309, -0.5669,  0.4439, -0.4753, -0.0873,  0.0265, -0.5976,
#         -0.0388, -0.8068,  1.0560,  0.1182,  0.3832, -0.6784,  0.3612,  0.3288,
#          0.3778,  0.8789, -0.4533, -1.0225, -0.2161, -0.0779, -1.4049, -0.6459,
#          2.2251, -0.9339,  1.5322,  0.1074, -0.7073,  0.8982,  1.2360, -0.0968,
#          0.7310, -0.3348,  0.0274, -0.5053,  1.1832, -2.0549, -0.7652, -0.9254,
#          1.2167, -0.0393, -0.2366,  0.9378,  0.3727,  1.6377, -1.6947,  0.4398,
#          0.3798, -0.6906, -0.5012, -0.4418]


# int16貼りつけ
# z = [
#     501, -167, 297, -137, -224, 170, -238, 418, -184, -180,
#     483, 26, -190, 510, -21, -277, 324, -8, 185, 290,
#     -186, -170, 141, 30, 274, 257, -120, -320, 87, -161,
#     -404, 62, 65, -195, -121, 87, -253, 361, -9, -316,
#     318, -545, 446, 222, 62, -367, 453, 77, -461, 136,
#     -26, -19, 202, 271, 207, -16, 210, 166, 51, 111,
#     497, 53, 185, 17, -336, 131, -173, -79, 278, -340,
#     -75, -477, 197, -12, -210, 27, 221, -586, 204, 385,
#     -188, -42, -292, -40, -287, 369, -30, 76, -347, 111,
#     219, 4, -40, -659, -167, -168, -8, -127, -29, -76
# ]
# z = torch.tensor(z, dtype=torch.int16)


## int16に変換
z = torch.tensor(z, dtype=torch.float32)
z = float_to_int16_torch(z)
print(z)

# G、B画像を作りたいとき、このベクトルに切り替える
#z = z+v1
#z = z+v2


pix_name =  "output_G.png"


tensor([ -33,  -94,  167,  291,  218,  -99,   32,  -39,  361, -273,   88,   45,
        -107, -208, -192, -194, -203,   79,  142,   96, -281,  -11,  -80,  372,
         -86, -144,  -51, -271,  222,   64,  -99, -247,  -30, -362,  118,   55,
        -386, -322, -337,   79,   43,  -28,   45,  106,  -25,  283,  179, -360,
         300,  108,  107, -131, -429,  -32,  373,  236, -250,  233,  205,  -50,
         -40,  142,   -1,  115, -394,  421,  251,  267, -181,   27,  177,  163,
         196,  -27,  -25,  277,  195, -129, -449,  184,  -62,   99,  -54, -269,
         374,  404,  -21,  202,  351,    0,  393, -270, -121,  191,  -13, -236,
        -414,  158,  254,  -19], dtype=torch.int16)


v1、v2、zを保存する場合

In [ ]:
import os
# z,v1,v2の保存先としてますが、最後の画像出力の保存先もここで指定してます。
save_dir_z = "report_soft_tanhtable_5"
os.makedirs(save_dir_z, exist_ok=True)

def save_int16_tensor_to_txt(tensor, path):
    tensor = tensor.detach().cpu().view(-1)  
    with open(path, "w") as f:
        for v in tensor:
            f.write(f"{int(v)}\n")


# z1 = z+v1
# z2 = z+v2


# save_int16_tensor_to_txt(z, os.path.join(save_dir_z, "z.txt"))
# save_int16_tensor_to_txt(z1, os.path.join(save_dir_z, "z1.txt"))
# save_int16_tensor_to_txt(z2, os.path.join(save_dir_z, "z2.txt"))
# save_int16_tensor_to_txt(v1, os.path.join(save_dir_z, "v1.txt"))
# save_int16_tensor_to_txt(v2, os.path.join(save_dir_z, "v2.txt"))


逆畳み込み演算
-

1層目の演算 net0_weight_shape = (100,512,4,4)
-


z100次元[100,1,1] 1×1の画像が100枚　　 z_1,z_2, .. , z_100

karnelは[100,512,4,4] 4×4の画像が512×100枚　　k_1,k_2,...,k_512

z_1*k_1 + z_2*k_1 + ... + z_100*k_1 = output[0]
output[511]

--------------------------------------------

### float演算用

In [278]:
# Q_FRAC_BITS = 7
# Q_SCALE = 1 << Q_FRAC_BITS #128

### カーネルの準備　kernel[1][1]などの形式で使えるように

kernel[0][0]～kernel[99][511]まで

In [279]:
path = os.path.join(OUTPUT_DIR, "G_q17_list", "net0_weight_q17_list.json")
with open(path, "r", encoding="utf-8") as f:
    w_list = json.load(f)

kernel_int8 = np.array(w_list, dtype=np.int8)
# print("kernel_int8",kernel_int8[99][511])
# print(kernel_int8.dtype, kernel_int8.min(), kernel_int8.max())

# kernel_int8 = kernel_int8.astype(np.int16)
# print("kernel_int8_int16",kernel_int8[99][511])

#print(kernel_int8.dtype, kernel_int8.min(), kernel_int8.max())
kernel = (kernel_int8.astype(np.int16) << 1)
print("kernel[0],[0]",kernel[99][511])

print(kernel.dtype, kernel.min(), kernel.max())
print(z.dtype, z.min(), z.max())

print("karnelshape", kernel.shape)
print("kernel[0],[0]",kernel[99][511])

kernel[0],[0] [[-30   0  -6  -4]
 [ 38   0  -4   0]
 [  0   2 -16 -10]
 [  8  -6  -6 -10]]
int16 -186 154
torch.int16 tensor(-953, dtype=torch.int16) tensor(939, dtype=torch.int16)
karnelshape (100, 512, 4, 4)
kernel[0],[0] [[-30   0  -6  -4]
 [ 38   0  -4   0]
 [  0   2 -16 -10]
 [  8  -6  -6 -10]]


### 1層目カーネルの計算　main

net0_calculateの計算　　z_1*k_[0][0] + z_2*k_[1][0] + ... + z_100*k_[100][0] = output[0]

100次元のzに対し、kernel[0][0]～kernel[100][0]を掛け合わせると、output0[1]になる。
これを512回行う。kernelは512×100枚ある。一列の計算で使用するのは100枚。

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝

net0_calculate_q78()内のint16、int32などの扱いについて

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝

int16 × int16 → int32(Q14.16)

Q14.16の値　＝　実数値　×　2^16

Q14.16をQ7.8に戻すためには、2^8で割りたい　（ビット演算形式： >>8 ）

ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

ダメな例）　①3.6(実数値) ×　2^16 = 235930(Q14.16)  　②235930>>8 = 921(Q7.8)　　③921 / 2^8 = 3.598(実数値)　ズレが生じる

対策法：丸めこみ　②で割る前に2^8の半分（2^7）を足す。

　　　例）①3.6(実数値) ×　2^16 = 235930(Q14.16)  　②235930＋128 >>8 = 922(Q7.8)　　③922 / 2^8 = 3.601(実数値)　ズレ軽減

ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

⇒　acc = (acc + (1<<7)) >> 8 と　round( acc / 2^8 )　は同義

In [280]:
# def net0_calculate_float(z_q,kernel,N):
#     acc = np.zeros((4,4), dtype=np.int16)
#     #kernel = kernel[::-1,::-1].astype(np.int64)

#     for k in range(100):
#         prod = np.int16(z_q[k])*kernel[k][N].astype(np.int16)
#         acc += prod

#     return acc

def net0_calculate_q78(z,kernel,N):
    acc = np.zeros((4,4),dtype=np.int32)

    for k in range(100):
        acc += np.int32(z[k]) * np.int32(kernel[k][N])
    
    # accはQ14.16（int32）　：　acc = 実数値 × 2^16
    # 
    #print("accint32",acc)
    acc = (acc + (1<<7)) >> 8
    #print("accint16",acc)

    # int16に飽和して格納
    acc = np.clip(acc, -32768, 32767).astype(np.int16)
    return acc

In [281]:
output0 = np.zeros((512, 4, 4), dtype=np.int16)

print(kernel.dtype, kernel.min(), kernel.max())
print(z.dtype, z.min(), z.max())

# kernel = kernel.astype(np.int32)
# print(kernel.dtype, kernel.min(), kernel.max())
# z = np.int32(z)
# print(z.dtype, z.min(), z.max())

for i in range(512):
     output0[i] = net0_calculate_q78(z,kernel,i)

# bit幅の検討
print("output0 min:", output0.min()) # -13.46
print("output0 max:", output0.max()) # 12.44

# ➡　候補１　Q6.9(16bit)  範囲±32  分解能1/512
# ➡　候補２　Q5.10(16bit) 範囲±16　分解能1/1024


# float演算の時との値の確認（OK）
# output0 = int16_to_float_np(output0)
# print("output0 min:", output0.min()) # -3.23
# print("output0 max:", output0.max()) # 3.2

output0_relu = np.maximum(output0, 0).astype(np.int16)

output0 = output0_relu

print("output0 min:", output0.min()) # 0
print("output0 max:", output0.max()) # 12.44

print("output0[0]",output0[0])
print("output0[1]",output0[1])
print("output0[1]",output0[511])

int16 -186 154
torch.int16 tensor(-953, dtype=torch.int16) tensor(939, dtype=torch.int16)
output0 min: -3922
output0 max: 3548
output0 min: 0
output0 max: 3548
output0[0] [[2038  196    0  934]
 [   0    0    0    0]
 [   0    0    0    0]
 [ 195  126   81  379]]
output0[1] [[510   0   0   8]
 [228 134   0  13]
 [  0   0   0   0]
 [  0   0   0   0]]
output0[1] [[   0   36  199 1087]
 [  74    0    0  206]
 [   0   63    0    1]
 [   0  124    0   44]]


二層目　net2_weight_shape = (512,256,4,4)
-

ストライド処理（ 4×4 → 7×7 ）

In [282]:
def stride(kernel: np.ndarray, stride: int=2) -> np.ndarray:
    kH, kW = kernel.shape
    outH = (kH - 1) * stride + 1 # 7
    outW = (kW - 1) * stride + 1

    out = np.zeros((outH, outW), dtype=kernel.dtype)
    out[::stride, ::stride] = kernel
    return out

padding=1、kernel=4の分の0埋め。カーネル数-1分の０を追加し、padding分の0を削除。

今回は00で周囲を埋める。　7×7　→　11×11

In [283]:
def padding(kernel: np.ndarray, pad:int) -> np.ndarray:
    return np.pad(kernel,((pad,pad),(pad,pad)),mode="constant",constant_values=0)

###  入力の作成
　
１層目のoutput0[0]～output[511]が入力となる

input2[0]～input[511]にした

In [284]:
input2 = np.zeros((512, 11, 11), dtype=np.int16)

for i in range(512):
    kernel_up = stride(output0[i],stride=2)
    kernel_pad = padding(kernel_up,pad=2)
    input2[i] = kernel_pad

print("input2[1]",input2[0])
print("input2.shape",input2[1].shape)

input2[1] [[   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0 2038    0  196    0    0    0  934    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0  195    0  126    0   81    0  379    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0]]
input2.shape (11, 11)


### 2層目のカーネルの取得

kernel2[0][0]～kernel2[511][255]

In [285]:
path = os.path.join(OUTPUT_DIR, "G_q17_list", "net1_weight_q17_list.json")
with open(path, "r", encoding="utf-8") as f:
    w_list = json.load(f)

kernel2_int8 = np.array(w_list, dtype=np.int8)
kernel2 = (kernel2_int8.astype(np.int16) << 1)

print("karnelshape", kernel2.shape)
print("kernel[0][0]",kernel2[0][0])

karnelshape (512, 256, 4, 4)
kernel[0][0] [[-10   8  18  10]
 [  2 -10 -26  10]
 [ -8   2   2  -2]
 [ -8  10  12  -4]]


### 畳み込み演算の準備

input2.shape           (512,11,11)

input_split2.shape    (512,64,4,4)

input_split2[0][0]  ～　input_split2[511][63]

In [286]:
def split_11(img11):
    patches = np.zeros((64,4,4),dtype=img11.dtype)
    idx = 0
    for y in range(8):
        for x in range(8):
            patches[idx] = img11[y:y+4,x:x+4]
            idx += 1
    return patches

input_split2 = np.zeros((512,64,4,4),dtype=input2.dtype)

for N in range(0,512):
    input_split2[N] = split_11(input2[N])

# 確認
print(input2.shape)           # (512,11,11)
print(input_split2.shape)     # (512,64,4,4)

print("first patch (top-left):")
print(input_split2[0][0])

print("last patch (bottom-right):")
print(input_split2[511][63])

(512, 11, 11)
(512, 64, 4, 4)
first patch (top-left):
[[   0    0    0    0]
 [   0    0    0    0]
 [   0    0 2038    0]
 [   0    0    0    0]]
last patch (bottom-right):
[[ 0  0  0  0]
 [ 0 44  0  0]
 [ 0  0  0  0]
 [ 0  0  0  0]]


### 畳み込み演算
input2[1]の分割input_split2[1][0]～input_split[1][63]に対して、kernel2[0][0]をかけていく。

input2[i]に対して、kernel2[i][0]をかけていく。



In [287]:
def conv4_int16(split: np.ndarray, kernel:np.ndarray) -> np.int16:
    acc = np.int32(0)

    k = kernel[::-1,::-1].astype(np.int32)
    p = split.astype(np.int32)

    for y in range(4):
        for x in range(4):
            acc += p[y,x]*k[y,x]
    acc = (acc + (1 << 7)) >> 8
    acc = np.clip(acc, -32768, 32767)

    return np.int16(acc)

一列分の総和をとる。

In [288]:
def sum_out_int16(out_q8_all):
    acc = np.sum(out_q8_all.astype(np.int16), axis=0)
    return acc.astype(np.int16)

def relu_int16(x:np.ndarray) -> np.ndarray:
    return np.maximum(x,0).astype(np.int16)

今までの考え方を統合したコード

In [289]:
output2 = np.zeros((256,8,8),dtype=np.int16)
out_q8_all = np.zeros((512,8,8), dtype=np.int16)

for K in range(0,256):
    for N in range(0,512):
        out_q = np.zeros(64, dtype=np.int16)

        for i in range(64):
            out_q[i] = conv4_int16(input_split2[N][i], kernel2[N][K])

        out_q8_all[N] = out_q.reshape(8,8)
    output2[K] = sum_out_int16(out_q8_all)

print("output2 min:", output2.min()) # -47.34
print("output2 max:", output2.max()) # 20.94

# bit幅の計算
# 候補１　Q7.8(16bit) 範囲[-128,127.996] 分解能　1/256
# 候補２　Q6.9(16bit) 範囲[-64,63.998]   分解能　1/512


output2_relu = relu_int16(output2)
output2 = output2_relu

print(output2[0])
print(output2[255])

output2 min: -12059
output2 max: 5729
[[   0    0    0    0    0    0    0    0]
 [   0    0  454  208  406    0    0  174]
 [3848    0    0  328    0 1341    0 2867]
 [   0    0 1495 4259  977    0   11 1139]
 [2110    0    0  390    0    0    0    0]
 [   0    0  877 1266 1904  336  194  440]
 [   0    0    0    0    0    0    0    0]
 [   0    2  243  387 1163    0    0  101]]
[[   0    0    0    0    0    0    0    0]
 [ 154    0 1086    0    0    0  671    0]
 [   0    0    0    0    0    0    0    0]
 [1492    0  269    0    0    0    0    0]
 [   0    0    0 1602    0    0    0    0]
 [ 112    0 1020    0    0    0  570    0]
 [   0    0  137 2281 1122 1317    0  625]
 [   0    0    0    0    0    0    0    0]]


3層目　net3_weight_shape = (256,128,4,4)
-


### 入力の作成   

input3 (256, 19, 19)

In [290]:
input3 = np.zeros((256, 19, 19), dtype=np.int16)

for i in range(256):
    kernel_up = stride(output2[i],stride=2)
    kernel_pad = padding(kernel_up,pad=2)
    input3[i] = kernel_pad

# print("input3[0]",input3[0])
# print("input3[255]",input3[255])
# print("input3.shape",input3.shape)

### 3層目のカーネルの取得

kernel3[0][0]～kernel3[255][127]

In [291]:
path = os.path.join(OUTPUT_DIR, "G_q17_list", "net2_weight_q17_list.json")
with open(path, "r", encoding="utf-8") as f:
    w_list = json.load(f)

kernel3_int8 = np.array(w_list, dtype=np.int8)
kernel3 = (kernel3_int8.astype(np.int16) << 1)

print("karnelshape", kernel3.shape)
print("kernel[0][0]",kernel3[0][127])

karnelshape (256, 128, 4, 4)
kernel[0][0] [[  0   0  -4 -18]
 [  0  14  -6   0]
 [ -2  -8   8  -2]
 [ 18  -2  -8   6]]


### 畳み込み演算の準備

In [292]:
def split_19(img19):
    patches = np.zeros((256,4,4),dtype=img19.dtype)
    idx = 0
    for y in range(16):
        for x in range(16):
            patches[idx] = img19[y:y+4,x:x+4]
            idx += 1
    return patches

input_split3 = np.zeros((256,256,4,4),dtype=input3.dtype) # 256枚のカーネルがあり、それが256分割になる

for N in range(0,256):
    input_split3[N] = split_19(input3[N])

# 確認
print(input3.shape)           
print(input_split3.shape)     

print("first patch (top-left):")
print(input_split3[0][0])

print("last patch (bottom-right):")
print(input_split3[255][255])

(256, 19, 19)
(256, 256, 4, 4)
first patch (top-left):
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
last patch (bottom-right):
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


### 畳み込み演算

In [293]:
output3 = np.zeros((128,16,16),dtype=np.int16)
out_q8_all = np.zeros((256,16,16), dtype=np.int16)

for K in range(0,128):
    for N in range(0,256):
        out_q = np.zeros(256, dtype=np.int16)

        for i in range(256):
            out_q[i] = conv4_int16(input_split3[N][i], kernel3[N][K])

        out_q8_all[N] = out_q.reshape(16,16)
    output3[K] = sum_out_int16(out_q8_all)

print("output3 min:", output3.min()) # -47.62
print("output3 max:", output3.max()) # 14.70

# bit幅の計算
# 候補１　Q7.8(16bit) 範囲[-128,127.996] 分解能　1/256
# 候補２　Q6.9(16bit) 範囲[-64,63.998]   分解能　1/512

output3_relu = relu_int16(output3)
output3 = output3_relu

#print(output3[0])
print(output3[127])

output3 min: -8292
output3 max: 4546
[[ 653    0    0    0    0    0 2025  442    0    0  270    0  134    0
     0    0]
 [2334  452  339    0    0    0    0    0    0 1061  519    0  674    0
  2686  514]
 [1720    0 3708    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [1351    0    0    0    0 1070    0    0    0  290    0    0    0    0
     0  442]
 [ 159    0    0    0    0    0    0    0    0    0    0    0    0    0
     0   88]
 [ 884  530    0    0    0    0    0    0    0  417    0    0    0    0
     0  664]
 [ 221    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [ 827    0  780    0  708 1170    0    0    0    0    0    0    0  635
     0   24]
 [  87  335    0    0    0    0    0    0    0  863    0    0    0    0
  1205  623]
 [ 472  127    0    0    0    0    0    0    0   29    0    0    0    0
     0    0]
 [ 797    0  174    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [ 864    0    0    0    0 1

4層目　 net4_weight_shape = (128,1,4,4)
-

### 入力の作成
input4 (128, 35, 35)

In [294]:

input4 = np.zeros((128, 35, 35), dtype=np.int16)

for i in range(128):
    kernel_up = stride(output3[i],stride=2)
    kernel_pad = padding(kernel_up,pad=2)
    input4[i] = kernel_pad

print("input4[0]",input4[0])
print("input4[127]",input4[127])
print("input4.shape",input4.shape)

input4[0] [[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0 608 ...   0   0   0]
 ...
 [  0   0   0 ...  66   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
input4[127] [[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0 653 ...   0   0   0]
 ...
 [  0   0 560 ... 446   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
input4.shape (128, 35, 35)


### 4層目のカーネルの取得

kerne4[0][0]～kernel4[127][0]

In [295]:
path = os.path.join(OUTPUT_DIR, "G_q17_list", "net3_weight_q17_list.json")
with open(path, "r", encoding="utf-8") as f:
    w_list = json.load(f)

kernel4_int8 = np.array(w_list, dtype=np.int8)
kernel4 =  (kernel4_int8.astype(np.int16) << 1)

print("karnelshape", kernel4.shape)
print("kernel[0][0]",kernel4[127][0])

karnelshape (128, 1, 4, 4)
kernel[0][0] [[ -2  12 -30 -26]
 [ -2  -6   6 -34]
 [ 12   4   8   6]
 [ -2   8  14   2]]


### 畳み込み演算の準備

In [296]:
def split_35(img35):
    patches = np.zeros((1024,4,4),dtype=img35.dtype)
    idx = 0
    for y in range(32):
        for x in range(32):
            patches[idx] = img35[y:y+4,x:x+4]
            idx += 1
    return patches

input_split4 = np.zeros((128,1024,4,4),dtype=input4.dtype) # 128枚のカーネルがあり、それが1024分割になる

for N in range(0,128):
    input_split4[N] = split_35(input4[N])

# 確認
print(input4.shape)           
print(input_split4.shape)     

print("first patch (top-left):")
print(input_split4[0][0])

print("last patch (bottom-right):")
print(input_split4[127][1023])

(128, 35, 35)
(128, 1024, 4, 4)
first patch (top-left):
[[  0   0   0   0]
 [  0   0   0   0]
 [  0   0 608   0]
 [  0   0   0   0]]
last patch (bottom-right):
[[  0   0   0   0]
 [  0 446   0   0]
 [  0   0   0   0]
 [  0   0   0   0]]


### 畳み込み演算

In [297]:
output4 = np.zeros((1,32,32),dtype=np.int16)
out_q8_all = np.zeros((128,32,32), dtype=np.int16)

K=0
for N in range(0,128):
    out_q = np.zeros(1024, dtype=np.int16)
    for i in range(1024):
        out_q[i] = conv4_int16(input_split4[N][i], kernel4[N][K])
    out_q8_all[N] = out_q.reshape(32,32)
output4[K] = sum_out_int16(out_q8_all)


print(output4.shape)
print(output4)
print(np.min(output4),np.max(output4)) #-2169  2874   -8.473～11.226
np.save("output4.npy", output4)


(1, 32, 32)
[[[-1091 -1694 -2655 ...  -888 -1289 -1785]
  [-1290  -904 -1072 ... -1372 -1808 -2383]
  [-1424 -1216 -1733 ... -1978 -2024 -2459]
  ...
  [ -812  -660  -636 ... -1541 -1466 -1466]
  [-1065  -975  -613 ... -1142 -1137 -1416]
  [-1264 -1750 -1284 ... -1302 -1732 -1533]]]
-4824 977


int16 Q7.8 →　int8 Q5.2に変換

In [298]:
# def round_shift_right(x:np.ndarray, rshift:int) -> np.ndarray:
#     x = x.astype(np.int32)
#     add = 1 << (rshift - 1)
#     x = x + np.where(x >= 0, add, -add)
#     return (x>> rshift).astype(np.int32)

# def q78_to_q52(x_q78:np.ndarray) -> np.ndarray:
#     y = round_shift_right(x_q78,6)
#     y = np.clip(y,-64,63)
#     return y.astype(np.int8)




# output4_int8 = q78_to_q52(output4)
# print("output4_int8:min max",np.min(output4_int8),np.max(output4_int8))   # int8 -35 45  float-8.75～11.25

# def int8_to_float_np(x_q: np.ndarray) -> np.ndarray:
#     return x_q.astype(np.int16) / 4  #Q5.2  1/2^2の分解能

# np.savetxt("output4_int8.txt", output4_int8.reshape(-1), fmt="%.8f")

### tanh

頭の整理

int16(Q7.8)のとき  　 int16：2874    　➡ ÷256 ➡  float：11.226

÷64(=2^6)をすると　　  int16：44.9      ➡ ÷4   ➡　float:11.225    

Q5.2（int8としてみると-128～127の範囲。floatで考えたとき、整数は-32～32の範囲、少数部は1/4=0.25の分解能）

In [299]:
# Q7.8からQ5.2に変換
output4_int8 = (output4 + (1<<5)) >> 6      # 小数部分を6bit分小さくするために2^6で除す
output4_int8 = np.clip(output4_int8,-128,127) # int8なので-128～127の値まででクリップ



### tanhテーブルの作成
v = (np.arange(256, dtype=np.int16) - 128)        # int8の値の全種類を生成：-128～127
x = v.astype(np.float32) / 4.0                    # int8がQ5.2のfloat値だと-32～32 に対応  ※tanhに入れるためにfloat化している
print("v",v)
print("x",x)

### tanhテーブル(in 1～256 ➡ out -128～127)
ans = np.rint(np.tanh(x) * 128.0) # -1～1の値を128倍し、rintで最も近い整数に丸める
lut_q17 = np.clip(ans, -128, 127).astype(np.int16) # -128～127のtanhテーブルを作る。インデクスでいうと、1～256に値が格納
                                                   # id=1のとき、lutq17=-128  id=256のとき、lutq17=127  ## idが入力するint8の値、lutq17が対応したtanhの値

v [-128 -127 -126 -125 -124 -123 -122 -121 -120 -119 -118 -117 -116 -115
 -114 -113 -112 -111 -110 -109 -108 -107 -106 -105 -104 -103 -102 -101
 -100  -99  -98  -97  -96  -95  -94  -93  -92  -91  -90  -89  -88  -87
  -86  -85  -84  -83  -82  -81  -80  -79  -78  -77  -76  -75  -74  -73
  -72  -71  -70  -69  -68  -67  -66  -65  -64  -63  -62  -61  -60  -59
  -58  -57  -56  -55  -54  -53  -52  -51  -50  -49  -48  -47  -46  -45
  -44  -43  -42  -41  -40  -39  -38  -37  -36  -35  -34  -33  -32  -31
  -30  -29  -28  -27  -26  -25  -24  -23  -22  -21  -20  -19  -18  -17
  -16  -15  -14  -13  -12  -11  -10   -9   -8   -7   -6   -5   -4   -3
   -2   -1    0    1    2    3    4    5    6    7    8    9   10   11
   12   13   14   15   16   17   18   19   20   21   22   23   24   25
   26   27   28   29   30   31   32   33   34   35   36   37   38   39
   40   41   42   43   44   45   46   47   48   49   50   51   52   53
   54   55   56   57   58   59   60   61   62   63   64   65   66   67
   6

In [300]:
#一時保存用

# ### tanhテーブル(in 1～256 ➡ out -128～127)
# ans = np.rint(np.tanh(x) * 128.0) # -1～1の値を128倍し、rintで最も近い整数に丸める
# lut_q17 = np.clip(ans, -128, 127).astype(np.int16) # -128～127のtanhテーブルを作る。インデクスでいうと、1～256に値が格納
#                                                   # id=1のとき、lutq17=-128  id=256のとき、lutq17=127  ## idが入力するint8の値、lutq17が対応したtanhの値
# #np.savetxt("lut_q17.txt",lut_q17.reshape(-1),fmt="%d")


# ### output4 ➡ 1～256の値に変換しidx作成
# idx = output4_int8.astype(np.int16) + 128    # idx = 入力するint8の値（-128～127）を正の1～256の値に


# # output4_tanh_table = lut_q17[idx]  / 128     # 対応するtableの値(-128～127)の値を-1～1の値に変換
# # np.savetxt("output4_tanh_table.txt",output4_tanh_table.reshape(-1),fmt="%.2f")
# # print(output4_tanh_table.shape)

# ### tanh(output4のidx) =  -128～127の値　　：　これに128を足して画像生成
# output4_tanh_table_img = (lut_q17[idx] + 128).clip(0,255)
# #np.savetxt("output4_tanh_table_img.txt",output4_tanh_table_img.reshape(-1),fmt="%.2f")
# img_u8 = output4_tanh_table_img[0].astype(np.uint8)


# ### 入力to画像直接変換テーブル保存
# u8 = np.clip(lut_q17 + 128, 0, 255).astype(np.uint8)
# img_u8_table = u8[idx]
# #np.savetxt("output4_tanh_u8_table.txt",u8.reshape(-1),fmt="%.2f")


# # img_q78 = output4_tanh_table[0]   
# # img_u8 = ((img_q78 + 1.0)*127.5).clip(0,255).astype(np.uint8)

# from PIL import Image
# import os
# save_dir = "int16_int8_tanh_table"
# os.makedirs(save_dir, exist_ok=True)

# # 
# save_path = os.path.join(save_dir, "output_G.png")
# Image.fromarray(img_u8, mode='L').save(save_path)


# save_path_table = os.path.join(save_dir, "output_G_table.png")
# Image.fromarray(img_u8_table[0], mode='L').save(save_path_table)


In [301]:
from PIL import Image
import os
# save_dir = "report_soft_tanhtable_2"
# os.makedirs(save_dir, exist_ok=True)



### output4 ➡ 1～256の値に変換しidx作成
idx = output4_int8.astype(np.int16) + 128    # idx = 入力するint8の値（-128～127）を正の1～256の値に

### tanh(output4のidx) =  -128～127の値　　：　これに128を足して画像生成
output4_tanh_table_img = (lut_q17[idx] + 128).clip(0,255)
img_u8 = output4_tanh_table_img[0].astype(np.uint8)


save_path = os.path.join(save_dir_z,pix_name)
Image.fromarray(img_u8, mode='L').save(save_path)

# ### 入力to画像直接変換テーブル保存
# u8 = np.clip(lut_q17 + 128, 0, 255).astype(np.uint8)
# img_u8_table = u8[idx]

# save_path_table = os.path.join(save_dir_z, "output_R_table.png")
# Image.fromarray(img_u8_table[0], mode='L').save(save_path_table)


floatにしたとき

In [302]:
# #output4 = int16_to_float_np(output4)
# output4 = int8_to_float_np(output4_int8)

# output4 = np.tanh(output4)
# print(output4.shape)
# print(output4)
# np.savetxt("output4.txt", output4.reshape(-1), fmt="%.8f")



画像出力
-


int16

In [303]:
# from PIL import Image

# img_q78 = output4[0]   # (32, 32)

# img_u8 = ((img_q78 + 1.0)*127.5).clip(0,255).astype(np.uint8)


# import os
# save_dir = "int16_int8_tanh_table"
# os.makedirs(save_dir, exist_ok=True)

# save_path = os.path.join(save_dir, "output_B.png")

# Image.fromarray(img_u8, mode='L').save(save_path)


カラー化（RGBそれぞれを保存済み想定）

In [ ]:
# input_dir = "report_soft_tanhtable_5"
# output_path = os.path.join(input_dir, "output_color.png")

# img_R = Image.open(os.path.join(input_dir,"output_R.png")).convert("L")
# img_G = Image.open(os.path.join(input_dir,"output_G.png")).convert("L")
# img_B = Image.open(os.path.join(input_dir,"output_B.png")).convert("L")

# # Numpy
# R = np.array(img_R, dtype=np.uint8)
# G = np.array(img_G, dtype=np.uint8)
# B = np.array(img_B, dtype=np.uint8)

# # RGB合成
# rgb = np.stack([R,G,B], axis=2)
# img_RGB = Image.fromarray(rgb, mode="RGB")

# H,W = R.shape
# canvas = Image.new("RGB",(W*4,H))

# canvas.paste(img_R.convert("RGB"),(0*W,0))
# canvas.paste(img_G.convert("RGB"),(1*W,0))
# canvas.paste(img_B.convert("RGB"),(2*W,0))
# canvas.paste(img_RGB,             (3*W,0))

# canvas.save(output_path)